Build and run Jupyter Notebook server from `/fuzzylite` as:

```bash
make jupyter
```

In [1]:
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("fuzzylite-debug")

// Use this if built in release mode
// #pragma cling load("fuzzylite")

#include <fuzzylite/Headers.h>

In [2]:
fl::Engine* engine = fl::FllImporter().fromFile("../mamdani/ObstacleAvoidance.fll");

In [3]:
std::cout << engine->toString();

Engine: ObstacleAvoidance
InputVariable: obstacle
  enabled: true
  range: 0.000 1.000
  lock-range: false
  term: left Ramp 1.000 0.000
  term: right Ramp 0.000 1.000
OutputVariable: mSteer
  enabled: true
  range: 0.000 1.000
  lock-range: false
  aggregation: Maximum
  defuzzifier: Centroid 100
  default: nan
  lock-previous: false
  term: left Ramp 1.000 0.000
  term: right Ramp 0.000 1.000
RuleBlock: mamdani
  enabled: true
  conjunction: none
  disjunction: none
  implication: AlgebraicProduct
  activation: General
  rule: if obstacle is left then mSteer is right
  rule: if obstacle is right then mSteer is left

In [4]:
std::string status;
if (not engine->isReady(&status))
    throw fl::Exception("[engine error] engine is not ready:\n" + status, FL_AT);

In [5]:
fl::InputVariable* obstacle = engine->getInputVariable("obstacle");
fl::OutputVariable* steer = engine->getOutputVariable("mSteer");

int resolution = 10;
for (int i = 0; i <= resolution; ++i){
    float dx = obstacle->range() / resolution;
    fl::scalar location = obstacle->getMinimum() + i * dx;
    obstacle->setValue(location);
    engine->process();
    std::cout << "obstacle.input = " << fl::Op::str(location) << 
        " => " << "steer.output = " << fl::Op::str(steer->getValue()) << std::endl;
}

obstacle.input = 0.000 => steer.output = 0.667
obstacle.input = 0.100 => steer.output = 0.660
obstacle.input = 0.200 => steer.output = 0.638
obstacle.input = 0.300 => steer.output = 0.602
obstacle.input = 0.400 => steer.output = 0.554
obstacle.input = 0.500 => steer.output = 0.500
obstacle.input = 0.600 => steer.output = 0.446
obstacle.input = 0.700 => steer.output = 0.398
obstacle.input = 0.800 => steer.output = 0.362
obstacle.input = 0.900 => steer.output = 0.340
obstacle.input = 1.000 => steer.output = 0.333


In [6]:
delete engine;